# 📊 Repositório de Consultas SQL para Analistas de Dados: **Departamento ESG - Environmental (Meio Ambiente)**

ESG é um **conjunto de padrões e boas práticas que visa definir se uma empresa é socialmente consciente, sustentável e corretamente gerenciada**. Trata-se de uma forma de medir o desempenho de sustentabilidade de uma organização. A sigla, em inglês, reúne os três pilares desse movimento:
-	Environmental (meio ambiente)
-	Social
-	Governance (governança)

O critério ambiental inclui exigências nesse campo, como:
- a gestão de resíduos;
- a política de desmatamento (caso aplicável);
- o uso de fontes de energia renováveis pela empresa;

Além disso, o critério ambiental pode também se estender ao controle exercido pela empresa em terras que possui, se há ações para melhorar e preservar a biodiversidade, por exemplo.

Comentários sobre o Projeto:
- 🔍 Contribuições: Estamos abertos a contribuições da comunidade de analistas de dados! Se você tiver consultas SQL úteis que gostaria de compartilhar ou sugestões para melhorias, sinta-se à vontade para enviar um pull request. Juntos, podemos expandir e aprimorar este repositório para beneficiar toda a comunidade de análise de dados.
- 🎯 Sinta-se à vontade para explorar, utilizar e contribuir para este repositório, e que ele possa ser uma ferramenta valiosa em sua jornada como analista de dados!

# 📌 **Emissões de Gases de Efeito Estufa (GEE)**

**Descrição**:
Este KPI mede as emissões de gases de efeito estufa (GEE), expressas em toneladas de CO2 equivalente, provenientes das operações da empresa, especificamente do transporte de pedidos. A métrica considera os diferentes modos de transporte utilizados (carro, moto, caminhão) e calcula o total de emissões associadas às entregas.

**Importância para o departamento ESG e para a empresa**:
- **🌍 Sustentabilidade Ambiental**: Monitorar e reduzir as emissões de GEE é crucial para minimizar o impacto ambiental das operações da empresa.
- **📈 Conformidade Regulamentar**: Muitas jurisdições têm regulamentações rigorosas sobre emissões de GEE. Manter esses níveis baixos ajuda a empresa a permanecer em conformidade e evitar multas.
- **🛡️ Reputação e Marca**: Consumidores e investidores estão cada vez mais preocupados com as práticas ambientais das empresas. Demonstrar um compromisso com a redução das emissões pode melhorar a imagem da marca e atrair clientes e investidores.
- **💰 Eficiência Operacional**: Reduzir as emissões muitas vezes está associado a melhorias na eficiência operacional, como otimização de rotas e uso de veículos mais eficientes, resultando em economia de custos.

### Query SQL para KPI

Vamos assumir que temos uma tabela `pedidos` com as seguintes colunas relevantes:
- `id`: ID do pedido
- `modal`: Modal de transporte (carro, moto, caminhão)
- `distancia_km`: Distância percorrida pelo modal em quilômetros

Para calcular as emissões de CO2, utilizamos fatores de emissão específicos para cada modal:
- Carro: 0.12 kg CO2 por km
- Moto: 0.09 kg CO2 por km
- Caminhão: 0.25 kg CO2 por km

Esses fatores de emissão são exemplos genéricos que podem ser usados para ilustrar o cálculo de emissões de CO2 com base em diferentes modais de transporte. No entanto, os valores reais podem variar dependendo de várias fontes e condições específicas. Esses valores são frequentemente utilizados em estudos e relatórios ambientais, mas é essencial utilizar dados específicos para a frota e condições operacionais da empresa. Recomendo consultar fontes como o IPCC (Intergovernmental Panel on Climate Change), a Agência Ambiental Europeia (EEA), ou bases de dados nacionais sobre emissões para obter fatores de emissão específicos e atualizados.

Aqui está a query SQL para calcular as emissões de CO2. Esta query primeiro calcula as emissões de CO2 em quilogramas para cada pedido, dependendo do modal de transporte, e depois soma todas as emissões, convertendo o total para toneladas de CO2 equivalente.:

```sql
WITH emissoes_por_pedido AS (
    SELECT
        id,
        modal,
        distancia_km,
        date_trunc('month', data_pedido) AS mes,
        CASE
            WHEN modal = 'carro' THEN distancia_km * 0.12  --- Fator de emissão para carro (kg CO2 por km)
            WHEN modal = 'moto' THEN distancia_km * 0.09   -- Fator de emissão para moto (kg CO2 por km)
            WHEN modal = 'caminhao' THEN distancia_km * 0.25  -- Fator de emissão para caminhão (kg CO2 por km)
        END AS emissao_co2_kg -- Emissões em kg
    FROM
        pedidos
),
emissoes_mensais AS (
    SELECT
        mes,
        SUM(emissao_co2_kg) / 1000 AS total_emissao_co2_ton -- Somando as emissões de CO2 calculadas para cada pedido e convertendo a unidade de quilogramas para toneladas
    FROM
        emissoes_por_pedido
    GROUP BY
        mes
),
emissoes_completas AS (
    SELECT
        mes,
        total_emissao_co2_ton,
        LAG(total_emissao_co2_ton) OVER (ORDER BY mes) AS emissao_mes_anterior,
        LAG(total_emissao_co2_ton, 12) OVER (ORDER BY mes) AS emissao_mes_ano_anterior
    FROM
        emissoes_mensais
)
SELECT
    mes,
    total_emissao_co2_ton,
    COALESCE(100 * (total_emissao_co2_ton - emissao_mes_anterior) / emissao_mes_anterior, 0) AS pct_mes_anterior,
    COALESCE(100 * (total_emissao_co2_ton - emissao_mes_ano_anterior) / emissao_mes_ano_anterior, 0) AS pct_mes_ano_anterior,
    COALESCE(emissao_mes_anterior, 0) * 0.9 AS meta,
    COALESCE(100 * total_emissao_co2_ton / (COALESCE(emissao_mes_anterior, 1) * 0.9), 0) AS pct_meta_batida
FROM
    emissoes_completas
ORDER BY
    mes;

```

### Explicação da Query

1. **CTE `emissoes_por_pedido`**:
   - **Objetivo**: Calcular as emissões de CO2 em quilogramas para cada pedido com base no modal de transporte e a distância percorrida.
   - **Colunas**:
     - `id`: ID do pedido.
     - `modal`: Modal de transporte (carro, moto, caminhão).
     - `distancia_km`: Distância percorrida pelo modal.
     - `mes`: Mês do pedido (truncado para o início do mês).
     - `emissao_co2_kg`: Emissões de CO2 em quilogramas calculadas com base no modal.

2. **CTE `emissoes_mensais`**:
   - **Objetivo**: Agregar as emissões de CO2 por mês e converter de quilogramas para toneladas.
   - **Colunas**:
     - `mes`: Mês.
     - `total_emissao_co2_ton`: Emissões totais de CO2 em toneladas para cada mês.

3. **CTE `emissoes_completas`**:
   - **Objetivo**: Calcular as emissões do mês anterior e do mesmo mês do ano anterior.
   - **Colunas**:
     - `mes`: Mês.
     - `total_emissao_co2_ton`: Emissões totais de CO2 em toneladas para cada mês.
     - `emissao_mes_anterior`: Emissões totais de CO2 em toneladas do mês anterior.
     - `emissao_mes_ano_anterior`: Emissões totais de CO2 em toneladas do mesmo mês no ano anterior.

4. **Seleção Final**:
   - **Objetivo**: Calcular os percentuais de emissão de CO2, meta de redução, e meta atingida.
   - **Colunas**:
     - `mes`: Mês.
     - `total_emissao_co2_ton`: Emissões totais de CO2 em toneladas para cada mês.
     - `pct_mes_anterior`: Percentual de crescimento ou queda em relação ao mês anterior.
     - `pct_mes_ano_anterior`: Percentual de crescimento ou queda em relação ao mesmo mês do ano anterior.
     - `meta`: Meta de emissões (90% da emissão do mês anterior).
     - `pct_meta_batida`: Percentual de meta atingida.


Essa query fornece uma visão clara das emissões de GEE ao longo do tempo, permitindo que a empresa monitore tendências e implemente estratégias para reduzir suas emissões de forma contínua.


# 📌 **Consumo de Energia: Quilowatts-hora (kWh) utilizados**

**Descrição**:
Este KPI mede o total de quilowatts-hora (kWh) utilizados pela empresa em um determinado período. Ele reflete o consumo de energia elétrica das operações da empresa, que é um indicador importante de eficiência energética e sustentabilidade ambiental.

**Importância para o departamento ESG e para a empresa**:
- **💡 Eficiência Energética**: Monitorar o consumo de energia ajuda a identificar áreas onde a eficiência pode ser melhorada, resultando em economia de custos.
- **🌱 Sustentabilidade Ambiental**: Reduzir o consumo de energia contribui para a redução da pegada de carbono da empresa, promovendo práticas mais sustentáveis.
- **📉 Conformidade Regulamentar**: Atender às normas de consumo energético e reduzir o uso de recursos naturais é crucial para a conformidade regulatória.
- **🏢 Reputação e Marca**: Empresas que demonstram responsabilidade ambiental através da gestão eficiente do consumo de energia são vistas de forma mais positiva por consumidores e investidores.

### Query SQL

Vamos assumir que temos uma tabela `consumo_energia` com os seguintes campos:
- `id`: ID do registro de consumo
- `data_consumo`: Data do consumo de energia
- `kwh_utilizados`: Quantidade de quilowatts-hora (kWh) consumidos

Aqui está a query SQL para calcular o consumo de energia:

```sql
WITH consumo_mensal AS (
    SELECT
        date_trunc('month', data_consumo) AS mes,
        SUM(kwh_utilizados) AS total_kwh_mensal
    FROM
        consumo_energia
    GROUP BY
        date_trunc('month', data_consumo)
),
consumo_anual AS (
    SELECT
        date_trunc('year', data_consumo) AS ano,
        SUM(kwh_utilizados) AS total_kwh_anual
    FROM
        consumo_energia
    GROUP BY
        date_trunc('year', data_consumo)
)
SELECT
    c.mes,
    c.total_kwh_mensal,
    a.total_kwh_anual
FROM
    consumo_mensal c
    JOIN consumo_anual a ON date_trunc('year', c.mes) = a.ano
ORDER BY
    c.mes;
```

### Explicação da Query

1. **CTE `consumo_mensal`**:
   - Agrupa o consumo de energia por mês e calcula o total de kWh consumidos mensalmente.

2. **CTE `consumo_anual`**:
   - Agrupa o consumo de energia por ano e calcula o total de kWh consumidos anualmente.

3. **Seleção Final**:
   - Junta os totais mensais e anuais do consumo de energia, permitindo a comparação e análise dos valores ao longo do tempo.



# 📌 **Consumo de Água: Metros cúbicos (m³) de água consumidos**

**Descrição**:
Este KPI mede o total de metros cúbicos (m³) de água consumidos pela empresa em um determinado período. Ele reflete o uso de água nas operações da empresa, que é um indicador importante de eficiência hídrica e sustentabilidade ambiental.

**Importância para o departamento ESG e para a empresa**:
- **🌍 Sustentabilidade Ambiental**: Monitorar e reduzir o consumo de água contribui para a conservação dos recursos hídricos e minimiza o impacto ambiental.
- **💧 Eficiência Hídrica**: Identificar áreas onde a eficiência no uso da água pode ser melhorada resulta em economia de custos e recursos.
- **📉 Conformidade Regulamentar**: Atender às normas de consumo hídrico e reduzir o uso de recursos naturais é crucial para a conformidade regulatória.
- **🏢 Reputação e Marca**: Empresas que demonstram responsabilidade ambiental através da gestão eficiente do consumo de água são vistas de forma mais positiva por consumidores e investidores.

### Query SQL

Vamos assumir que temos uma tabela `consumo_agua` com os seguintes campos:
- `id`: ID do registro de consumo
- `data_consumo`: Data do consumo de água
- `m3_utilizados`: Quantidade de metros cúbicos (m³) de água consumidos

Aqui está a query SQL para calcular o consumo de água:

```sql
WITH consumo_mensal AS (
    SELECT
        date_trunc('month', data_consumo) AS mes,
        SUM(m3_utilizados) AS total_m3_mensal
    FROM
        consumo_agua
    GROUP BY
        date_trunc('month', data_consumo)
),
consumo_anual AS (
    SELECT
        date_trunc('year', data_consumo) AS ano,
        SUM(m3_utilizados) AS total_m3_anual
    FROM
        consumo_agua
    GROUP BY
        date_trunc('year', data_consumo)
)
SELECT
    c.mes,
    c.total_m3_mensal,
    a.total_m3_anual
FROM
    consumo_mensal c
    JOIN consumo_anual a ON date_trunc('year', c.mes) = a.ano
ORDER BY
    c.mes;
```

### Explicação da Query

1. **CTE `consumo_mensal`**:
   - Agrupa o consumo de água por mês e calcula o total de m³ consumidos mensalmente.

2. **CTE `consumo_anual`**:
   - Agrupa o consumo de água por ano e calcula o total de m³ consumidos anualmente.

3. **Seleção Final**:
   - Junta os totais mensais e anuais do consumo de água, permitindo a comparação e análise dos valores ao longo do tempo.

# 📌 **Gestão de Resíduos: Toneladas de resíduos gerados e reciclados**

**Descrição**:
Este KPI mede a quantidade total de resíduos gerados e reciclados pela empresa em um determinado período, em toneladas. Ele reflete a eficácia das práticas de gestão de resíduos e sustentabilidade ambiental da empresa.

**Importância para o departamento ESG e para a empresa**:
- **🌍 Sustentabilidade Ambiental**: Monitorar e reduzir a quantidade de resíduos gerados e aumentar a quantidade reciclada contribui para a preservação ambiental.
- **♻️ Economia Circular**: Implementar práticas de reciclagem promove a economia circular, reduzindo a necessidade de novos recursos.
- **📉 Conformidade Regulamentar**: Atender às normas de gestão de resíduos e práticas de reciclagem é crucial para a conformidade regulatória.
- **🏢 Reputação e Marca**: Empresas que demonstram responsabilidade ambiental através da gestão eficiente de resíduos são vistas de forma mais positiva por consumidores e investidores.

### Query SQL

Vamos assumir que temos uma tabela `gestao_residuos` com os seguintes campos:
- `id`: ID do registro de resíduos
- `data_registro`: Data do registro de resíduos
- `tipo_residuo`: Tipo de resíduo (gerado ou reciclado)
- `toneladas`: Quantidade de resíduos em toneladas

Para calcular a quantidade total de resíduos gerados e reciclados, agregamos os resíduos por tipo, mês e ano.

Aqui está a query SQL para calcular a gestão de resíduos:

```sql
WITH residuos_mensais AS (
    SELECT
        date_trunc('month', data_registro) AS mes,
        tipo_residuo,
        SUM(toneladas) AS total_toneladas_mensal
    FROM
        gestao_residuos
    GROUP BY
        date_trunc('month', data_registro), tipo_residuo
),
residuos_anuais AS (
    SELECT
        date_trunc('year', data_registro) AS ano,
        tipo_residuo,
        SUM(toneladas) AS total_toneladas_anual
    FROM
        gestao_residuos
    GROUP BY
        date_trunc('year', data_registro), tipo_residuo
),
residuos_completos AS (
    SELECT
        r.mes,
        r.tipo_residuo,
        r.total_toneladas_mensal,
        a.total_toneladas_anual,
        COALESCE(
            CASE
                WHEN r.tipo_residuo = 'gerado' THEN r.total_toneladas_mensal
                ELSE 0
            END, 0
        ) AS total_gerado,
        COALESCE(
            CASE
                WHEN r.tipo_residuo = 'reciclado' THEN r.total_toneladas_mensal
                ELSE 0
            END, 0
        ) AS total_reciclado
    FROM
        residuos_mensais r
    JOIN residuos_anuais a ON date_trunc('year', r.mes) = a.ano AND r.tipo_residuo = a.tipo_residuo
),
residuos_agregados AS (
    SELECT
        mes,
        SUM(total_gerado) AS total_gerado_mensal,
        SUM(total_reciclado) AS total_reciclado_mensal
    FROM
        residuos_completos
    GROUP BY
        mes
)
SELECT
    rc.mes,
    rc.tipo_residuo,
    rc.total_toneladas_mensal,
    rc.total_toneladas_anual,
    ra.total_gerado_mensal,
    ra.total_reciclado_mensal,
    CASE
        WHEN ra.total_gerado_mensal > 0 THEN
            100 * (ra.total_gerado_mensal - ra.total_reciclado_mensal) / ra.total_gerado_mensal
        ELSE
            0
    END AS pct_falta_reciclar
FROM
    residuos_completos rc
JOIN
    residuos_agregados ra ON rc.mes = ra.mes
ORDER BY
    rc.mes, rc.tipo_residuo;
```

### Explicação da Query

1. **CTE `residuos_mensais`**:
   - Agrupa os resíduos por mês e tipo, calculando o total de toneladas mensais para cada tipo de resíduo (gerado e reciclado).

2. **CTE `residuos_anuais`**:
   - Agrupa os resíduos por ano e tipo, calculando o total de toneladas anuais para cada tipo de resíduo (gerado e reciclado).

3. **CTE `residuos_completos`**:
   - Junta os totais mensais e anuais dos resíduos, separando os resíduos gerados e reciclados em colunas diferentes para facilitar o cálculo.

4. **CTE `residuos_agregados`**:
   - Agrega os resíduos gerados e reciclados por mês.

5. **Seleção Final**:
   - Calcula o percentual do que falta para a quantidade de resíduos reciclados atingir a quantidade de resíduos gerados no mês.
  


# 📌 **Uso de Recursos Renováveis: Percentual de energia renovável no consumo total**

**Descrição**:
Este KPI mede o percentual de energia renovável utilizada em relação ao consumo total de energia da empresa. Ele reflete o compromisso da empresa com práticas sustentáveis e a transição para fontes de energia renovável.

**Importância para o departamento ESG e para a empresa**:
- **🌍 Sustentabilidade Ambiental**: Aumentar o uso de energia renovável ajuda a reduzir a pegada de carbono e promove a sustentabilidade ambiental.
- **💡 Eficiência Energética**: Transitar para fontes de energia renovável pode resultar em economia a longo prazo e maior independência energética.
- **📉 Conformidade Regulamentar**: Atender a regulamentações e metas de sustentabilidade é crucial para a conformidade regulatória.
- **🏢 Reputação e Marca**: Empresas que adotam práticas de energia renovável são vistas de forma mais positiva por consumidores e investidores.

### Query SQL

Vamos assumir que temos uma tabela `consumo_energia` com os seguintes campos:
- `id`: ID do registro de consumo
- `data_consumo`: Data do consumo de energia
- `kwh_utilizados`: Quantidade de quilowatts-hora (kWh) consumidos
- `fonte_energia`: Fonte de energia utilizada (renovável ou não renovável)

Para calcular o percentual de energia renovável no consumo total, agregamos o consumo de energia por tipo, mês e ano.

Aqui está a query SQL para calcular o uso de recursos renováveis:

```sql
WITH energia_mensal AS (
    SELECT
        date_trunc('month', data_consumo) AS mes,
        fonte_energia,
        SUM(kwh_utilizados) AS total_kwh_mensal
    FROM
        consumo_energia
    GROUP BY
        date_trunc('month', data_consumo), fonte_energia
),
energia_anual AS (
    SELECT
        date_trunc('year', data_consumo) AS ano,
        fonte_energia,
        SUM(kwh_utilizados) AS total_kwh_anual
    FROM
        consumo_energia
    GROUP BY
        date_trunc('year', data_consumo), fonte_energia
),
energia_agregada_mensal AS (
    SELECT
        mes,
        SUM(total_kwh_mensal) FILTER (WHERE fonte_energia = 'renovável') AS kwh_renovavel_mensal,
        SUM(total_kwh_mensal) AS kwh_total_mensal
    FROM
        energia_mensal
    GROUP BY
        mes
),
energia_agregada_anual AS (
    SELECT
        ano,
        SUM(total_kwh_anual) FILTER (WHERE fonte_energia = 'renovável') AS kwh_renovavel_anual,
        SUM(total_kwh_anual) AS kwh_total_anual
    FROM
        energia_anual
    GROUP BY
        ano
)
SELECT
    em.mes,
    em.kwh_renovavel_mensal,
    em.kwh_total_mensal,
    (em.kwh_renovavel_mensal::float / em.kwh_total_mensal) * 100 AS pct_energia_renovavel_mensal,
    ea.kwh_renovavel_anual,
    ea.kwh_total_anual,
    (ea.kwh_renovavel_anual::float / ea.kwh_total_anual) * 100 AS pct_energia_renovavel_anual
FROM
    energia_agregada_mensal em
    JOIN energia_agregada_anual ea ON date_trunc('year', em.mes) = ea.ano
ORDER BY
    em.mes;
```

### Explicação da Query

1. **CTE `energia_mensal`**:
   - Agrupa o consumo de energia por mês e fonte de energia, calculando o total de kWh consumidos mensalmente para cada fonte de energia (renovável e não renovável).

2. **CTE `energia_anual`**:
   - Agrupa o consumo de energia por ano e fonte de energia, calculando o total de kWh consumidos anualmente para cada fonte de energia.

3. **CTE `energia_agregada_mensal`**:
   - Agrega o total de kWh mensais por fonte de energia, separando o consumo de energia renovável e o consumo total.

4. **CTE `energia_agregada_anual`**:
   - Agrega o total de kWh anuais por fonte de energia, separando o consumo de energia renovável e o consumo total.

5. **Seleção Final**:
   - Calcula o percentual de energia renovável no consumo total mensal e anual, permitindo a comparação e análise dos valores ao longo do tempo.

### Descrição das colunas

- **kwh_renovavel_mensal**: A quantidade de energia renovável consumida em kWh durante o mês.
- **kwh_total_mensal**: A quantidade total de energia consumida em kWh durante o mês.
- **pct_energia_renovavel_mensal**: O percentual de energia renovável consumida em relação ao consumo total durante o mês.
- **kwh_renovavel_anual**: A quantidade de energia renovável consumida em kWh durante o ano.
- **kwh_total_anual**: A quantidade total de energia consumida em kWh durante o ano.
- **pct_energia_renovavel_anual**: O percentual de energia renovável consumida em relação ao consumo total durante o ano.

Esses dados fornecem uma visão detalhada do uso de energia renovável pela empresa, tanto em termos absolutos quanto percentuais, permitindo uma análise eficaz das tendências e a identificação de oportunidades para aumentar o uso de recursos renováveis.
